In [101]:
import pandas as pd
import numpy as np
import json
import regex

In [109]:
with open("traducciones.json") as file:
    traducciones=json.load(file)

In [141]:
df = pd.read_excel("BPC_Ingredientes_ALL_borrador_updated.xlsx")
df = pd.read_excel("rararara.xlsx")
df = pd.read_excel("removiendo_harmonised.xlsx")
df = pd.read_excel("a_partir_de_ahora_es_manual_2.xlsx")

In [77]:
df

,Unnamed: 0,COSING Ref No,INCI name,INN name,Ph. Eur. Name,CAS No,EC No,NamexCas,NamexEC,NamexName,...,Ref.Env,Add Info.Env,Cancer.Risk,Development.Risk,Allergies.Risk,Endocryne.Risk,Prohibited.Risk,Env.Risk,Total.Risk,modificado
0,0,34582,IPBC,-,3-Iodo-2-propynyl butylcarbamate,55406-53-6,259-627-5,-,-,-,...,European Commission Legislation (Regulation (E...,Suspected hazardous to the aquatic environment...,1.0,0.0,2.0,0.0,1.0,2.0,5.1,1.0
1,1,36522,"PHENOXYETHANOL;2-Phenoxyethanol, Euxyl KÂ® 400...",-,phenoxyethanolum,122-99-6,204-589-7,-,-,-,...,-,-,0.0,0.0,1.0,0.0,1.0,0.0,2.0,NaN
2,3,32185,BHT,butylated hydroxytoluene,butylhydroxytoluenum,128-37-0,204-881-4,-,-,PHENYL PROPYL TRIMETHICONE/DIPHENYLMETHICONE,...,"L. Gue, What's Inside? That Counts. A Survey o...",-,3.0,0.0,1.0,1.0,0.0,2.0,4.3,NaN
3,4,32184,BHA,butylated hydroxyanisole,-,25013-16-5,246-563-8,-,-,-,...,NaN,-,0.0,3.0,0.0,0.0,0.0,0.0,3.0,NaN
4,5,37364,PVP,crospovidone,povidonum,9003-39-8,-,-,-,-,...,NaN,-,2.0,2.0,0.0,0.0,0.0,0.0,2.2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,30703,-,FRAGRANCIAS NATURALES,-,-,7397-46-8,-,-,-,-,...,-,-,0.0,0.0,3.0,0.0,0.0,0.0,3.0,1.0
6119,30704,-,sodium alkyl ethoxy sulfate,-,-,7397-46-8,-,-,-,-,...,-,-,0.0,2.0,0.0,0.0,0.0,1.0,3.0,1.0
6120,30705,-,Fd&C Amarillo 5,-,-,7397-46-8,-,-,-,-,...,-,-,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0
6121,30706,-,ALCOHOL GRASO POLIETOXILADO,-,-,7397-46-8,-,-,-,-,...,-,-,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0


In [142]:
c1=0
c2=0
lta=[]
los_que_tienen_har=pd.DataFrame()
variantes_harmonised=set()
df["MODIFICAR"] = "N"
for n,row in df.iterrows():
    if row["Info para Reporte"][:3].lower() == "- h":
        # los_que_tienen_har=los_que_tienen_har.append(row)
        shortened = row["Info para Reporte"][3:]
        df.loc[n,"MODIFICAR"]="Y"
        c1+=1
    elif row["Info para Reporte"][:3].lower() == "har":
        variantes_harmonised.add(row["Info para Reporte"])
        c1+=1
        df.loc[n,"MODIFICAR"]="Y"

    else:
        if "harmonised" in row["Info para Reporte"].lower():
            c2+=1
            lta.append(row["Info para Reporte"])
            df.loc[n,"MODIFICAR"]="Y"
          

In [125]:
c2

11

In [136]:
def parse_harmonised(string:str) -> str: 
    return regex.findall(r"(?<=Harmonised classification for\s)[\w\s]*", string)

def parse_suspected(string:str) -> str: 
    return regex.findall(r"(?<=Suspected\s)[\w\s]*", string)
    

In [114]:
r=parse_harmonised(df[df.MODIFICAR=="Y"].iloc[0,:]["Info para Reporte"])

In [137]:
dicts={k:parse_harmonised(k) for k in traducciones if parse_harmonised(k) != []}
dicts_s=dicts={k:parse_suspected(k) for k in traducciones if parse_suspected(k) != []}

In [118]:
c = 0
for key, values in dicts.items():
    if r == values:
        print("kawabunga")
        c+=1

kawabunga


In [138]:
for n,row in df.iterrows():
    if row.MODIFICAR == "Y":
        efectos=parse_harmonised(row["Info para Reporte"])
        for key, values in dicts.items():
            if set(efectos) == set(values):
                df.loc[n, "Info para Reporte"] = traducciones[key]

In [139]:
df.to_excel("a_partir_de_ahora_es_manual_2.xlsx") 

In [143]:
c1,c2

(190, 11)

In [134]:
set(["a","b","c"]) == set(["b","c","a"])

True

In [86]:
traducciones

NameError: name 'traducciones' is not defined

In [83]:
lta2=[]
for node in lta: 
    if "#" in node:
        lta2.append(node.split("#")[0])
    else: 
        lta2.append(node.split("Harmonised")[0])
        

In [128]:
for key in traducciones.keys():
    if "bioaccumulative" in key:
        print({key:traducciones[key]})

{'Suspected bioaccumulative/ Suspected hazardous to the aquatic environment/ Suspected skin sensitiser': 'Ingrediente potencialmente irritante para la piel, bioacumulable y con impacto negativo en la vida acuática.'}
{'Suspected bioaccumulative/ Suspected hazardous to the aquatic environment/ Suspected mutagen': 'Ingrediente con capacidad potencial de alterar la información genética de un organismo biológico, potencialmente bioacumulable y con impacto negativo en la vida acuática.'}
{'Suspected bioaccumulative/ Suspected hazardous to the aquatic environment/ Suspected mutagen/ Suspected persistent in the environment/ Suspected skin sensitiser': 'Ingrediente con capacidad potencial de alterar la información genética de un organismo biológico, potencialmente sensibilizador de la piel, bioacumulable, con impacto negativo en la vida acuática y con efectos duraderos en el ambiente.'}
{'Suspected bioaccumulative/ Suspected hazardous to the aquatic environment/ Suspected mutagen/ Suspected pe

In [85]:
lista = []
for node in variantes_harmonised:
    # lista += node.split("#")
    lista += node.split("/ ")
set(lista)

{'Harmonised classification for acute toxicity',
 'Harmonised classification for aquatic toxicity',
 'Harmonised classification for carcinogenicity',
 'Harmonised classification for eye damage',
 'Harmonised classification for eye irritation',
 'Harmonised classification for mutagenicity',
 'Harmonised classification for reprotoxicity',
 'Harmonised classification for respiratory sensitisation',
 'Harmonised classification for skin corrosion',
 'Harmonised classification for skin sensitisation',
 'Harmonised classification for specific target organ toxicity',
 'Potential Cholestasis inducer.',
 'Suspected bioaccumulative',
 'Suspected carcinogen',
 'Suspected hazardous to the aquatic environment',
 'Suspected mutagen',
 'Suspected persistent in the environment',
 'Suspected persistent in the environment ',
 'Suspected respiratory sensitiser',
 'Suspected skin irritant',
 'Suspected skin sensitiser',
 'Suspected to meet STOT RE classification',
 'Suspected toxic for reproduction'}

In [11]:
los_que_tienen_hari in df["Info para Reporte"].tuples()
    print(i)

SyntaxError: invalid syntax (3662199407.py, line 1)

In [12]:
dir(pd.Series)

['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__